# Section 5 - Markov Models

## Basic definitions

>**Markov property**:
- The probability of a word occurring depends only on the previous word or a fixed number of preceding words, and not on the earlier words.

>**State transition matrix (denoted as `A`)**:
- A 2D matrix of the probabilities of transitioning from one state to another.

>**Initial state distribution (denoted as `π`)**:
- The probability distribution over the possible starting states.

>**Priors**:
- Probability distribution of the classes.

## Case Study - Building a Text Classifier

In [ ]:
# Download datasets
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt


--2024-06-15 04:42:46--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.003s  

2024-06-15 04:42:46 (8.66 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2024-06-15 04:42:46--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP re

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [ ]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [ ]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [ ]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuations
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [ ]:
train_text, test_text, y_train, y_test = train_test_split(input_texts, labels)

In [ ]:
len(y_train), len(y_test)

(1615, 539)

In [ ]:
train_text[:5], y_train[:5]

(['you mean oh for some miss',
  'a passionate light such for his spirit was fit',
  'let my future radiant shine',
  'and then becoming reconciled',
  'science true daughter of old time thou art'],
 [1, 0, 0, 1, 0])

In [ ]:
idx = 1
word2idx = {'<unk>':0}

In [ ]:
# populate word2idx
for text in train_text:
  tokens = text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token] = idx
      idx += 1

In [ ]:
word2idx

{'<unk>': 0,
 'you': 1,
 'mean': 2,
 'oh': 3,
 'for': 4,
 'some': 5,
 'miss': 6,
 'a': 7,
 'passionate': 8,
 'light': 9,
 'such': 10,
 'his': 11,
 'spirit': 12,
 'was': 13,
 'fit': 14,
 'let': 15,
 'my': 16,
 'future': 17,
 'radiant': 18,
 'shine': 19,
 'and': 20,
 'then': 21,
 'becoming': 22,
 'reconciled': 23,
 'science': 24,
 'true': 25,
 'daughter': 26,
 'of': 27,
 'old': 28,
 'time': 29,
 'thou': 30,
 'art': 31,
 'perhaps': 32,
 'it': 33,
 'may': 34,
 'be': 35,
 'that': 36,
 'mind': 37,
 'is': 38,
 'wrought': 39,
 'i': 40,
 'say': 41,
 'on': 42,
 'consideration': 43,
 'know': 44,
 'one': 45,
 'didnt': 46,
 'thrive': 47,
 'the': 48,
 'west': 49,
 'getting': 50,
 'out': 51,
 'gold': 52,
 'save': 53,
 'but': 54,
 'soul': 55,
 'in': 56,
 'thine': 57,
 'uplifted': 58,
 'eyes': 59,
 'smell': 60,
 'wet': 61,
 'feathers': 62,
 'heat': 63,
 'supposed': 64,
 'to': 65,
 'mad': 66,
 'resurrection': 67,
 'deepburied': 68,
 'faith': 69,
 'will': 70,
 'leave': 71,
 'their': 72,
 'tatters': 73,
 

In [ ]:
len(word2idx)

2520

In [ ]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens] # to handle the fact that not all tokens in test test is present in word2idx
  test_text_int.append(line_as_int)

In [ ]:
train_text_int[100:105]

[[36, 391, 65, 35, 392, 393, 104],
 [56, 394, 115, 40, 395, 45, 164, 396, 48, 397],
 [398, 173, 252, 399, 86, 400, 401, 402],
 [20, 7, 403, 404],
 [65, 48, 405, 406, 229, 48, 407, 65, 48, 408]]

In [ ]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [ ]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last_idx
      last_idx = idx

compute_counts([t for t, y in zip(train_text_int, y_train) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, y_train) if y == 1], A1, pi1)

In [ ]:
# normalize A and pi so they are valid probability matrices
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [ ]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
# compute priors
count0 = sum(y == 0 for y in y_train)
count1 = sum(y == 1 for y in y_train)
total = len(y_train)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.32507739938080493, 0.6749226006191951)

In [ ]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx
    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                    for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [ ]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [ ]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == y_train)}")

Train acc: 0.9950464396284829


In [ ]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == y_test)}")

Test acc: 0.8070500927643784


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
cm_train = confusion_matrix(y_train, Ptrain)
cm_train

array([[ 517,    8],
       [   0, 1090]])

In [ ]:
cm_test = confusion_matrix(y_test, Ptest)
cm_test

array([[ 95,  98],
       [  6, 340]])

In [ ]:
f1_score(y_train, Ptrain), f1_score(y_test, Ptest)

(0.9963436928702011, 0.8673469387755102)

## Case Study - Poetry Generator

In [1]:
import numpy as np
import string

np.random.seed(1234)

In [8]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {}

In [3]:
def remove_punctuation(s):
  return s.translate(str.maketrans('', '', string.punctuation))

In [4]:
# Download data
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2024-06-15 10:49:43--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56286 (55K) [text/plain]
Saving to: ‘robert_frost.txt’

robert_frost.txt    100%[===================>]  54.97K  --.-KB/s    in 0.004s  

2024-06-15 10:49:44 (13.5 MB/s) - ‘robert_frost.txt’ saved [56286/56286]



In [5]:
def add2dict(d, k, v):
  if k not in d:
    d[k] = []
  d[k].append(v)

# [cat, cat, dog, dog, dog, dog, mouse, ...]

In [9]:
for line in open('robert_frost.txt'):
  tokens = remove_punctuation(line.rstrip().lower()).split()

  T = len(tokens)
  for i in range(T):
    t = tokens[i]
    if i == 0:
      # measure the distribution of the first word
      initial[t] = initial.get(t, 0.) + 1
    else:
      t_1 = tokens[i-1]
      if i == T - 1:
        # measure probabiltty of ending the line
        add2dict(second_order, (t_1, t), 'END')
      if i == 1:
        # measure distribution of the second word
        # given only first word
        add2dict(first_order, t_1, t)
      else:
        t_2 = tokens[i-2]
        add2dict(second_order, (t_2, t_1), t)

In [15]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
  initial[t] = c / initial_total

In [17]:
# convert [cat, cat, cat, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2pdict(ts):
  # turn each list of possibilities into a dictionary of probabilities
  d = {}
  n = len(ts)
  for t in ts:
    d[t] = d.get(t, 0.) + 1
  for t, c in d.items():
    d[t] = c / n
  return d

In [18]:
for t_1, ts in first_order.items():
  # replace list with dictionary of probabilities
  first_order[t_1] = list2pdict(ts)

In [20]:
for k, ts in second_order.items():
  second_order[k] = list2pdict(ts)

In [22]:
def sample_word(d):
  # print "d:", d
  p0 = np.random.random()
  # print "p0:", p0
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t

In [27]:
def generate():
  for i in range(4): # generate 4 lines
    sentence = []
    # initial word
    w0 = sample_word(initial)
    sentence.append(w0)

    # sample second word
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    # second order transitions until END
    while True:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2
    print(' '.join(sentence))

In [32]:
generate()

the origin of all this now too much
though what a gentle lot we are
however far you must be near the window toward the light
up to it and were off to find or force a strait
